In [19]:
import boto3
from datetime import datetime, timedelta 

client = boto3.client('ec2')


In [20]:
instances = client.describe_instances()
instances['Reservations']

yesterday = (datetime.now() - timedelta(days=2)).timestamp()

reservations = instances['Reservations']

instances_over_48hours_old = list(
    filter(lambda instance: instance['Instances'][0]['LaunchTime'].timestamp() < yesterday, reservations)
)

In [21]:
save_list = []
kill_list = []

for ins in instances_over_48hours_old:
    instance_name = None
    instance_venue = None
    instance_project = None
    
    for k in ins['Instances'][0]['Tags']:
        if k['Key'] == 'Name':
          instance_name = k['Value']
        if k['Key'] == 'Venue':
           instance_venue = k['Value'] 
        if k['Key'] == 'Proj':
           instance_project = k['Value'] 
    if instance_project in ['unity'] and instance_venue in ['venue-dev', 'dev']:
        save_list.append(ins['Instances'][0]['InstanceId'])
        print ("SAVE {} - {} ({}|{})".format(ins['Instances'][0]['InstanceId'], instance_name, instance_project, instance_venue))
    elif "Management Console" in instance_name:
        save_list.append(ins['Instances'][0]['InstanceId'])
        print ("SAVE {} - {} ({}|{})".format(ins['Instances'][0]['InstanceId'], instance_name, instance_project, instance_venue))
    else:
        kill_list.append(ins['Instances'][0]['InstanceId'])
        print ("KILL {} - {} ({}|{})".format(ins['Instances'][0]['InstanceId'], instance_name, instance_project, instance_venue))

SAVE i-0bb1b86acf0dbfb26 - jupyter (unity|venue-dev)
SAVE i-0fe982b45c0eff45a - defaultGroup (unity|dev)
KILL i-053272eefc121963c - defaultGroup (ramesh-2|dev)
SAVE i-0866cb5bfdbf78adc - Unity Management Console (ramesh-3/dev) (ramesh-3|dev)
KILL i-0b5bad8bbb22f44b0 - ramesh-3-dev-sps-karpenter (ramesh-3|dev)
SAVE i-0cf806754945857d6 - Unity Management Console (btl-dc6/dev) (btl-dc6|dev)
SAVE i-02be479b5204c8b4d - unity-dev-sps-karpenter (unity|dev)
SAVE i-0ee1f2dd75b0b15da - unity-dev-sps-karpenter (unity|dev)
KILL i-0176693bd16a72214 - ramesh-3-dev-sps-karpenter (ramesh-3|dev)
KILL i-0d75e4a3eba3999ae - ramesh-3-dev-sps-karpenter (ramesh-3|dev)
SAVE i-07598f0de69d04498 - Unity Management Console (graceal-6/dev) (graceal-6|dev)
KILL i-0540726b76a61e9ef - btl-dc8-dev-sps-karpenter (btl-dc8|dev)
SAVE i-05dfbc59fc7ebbec5 - jupyter (unity|venue-dev)
SAVE i-0db4a545e8a334023 - Unity Management Console (unity/dev) (unity|dev)
SAVE i-019104bc5ee06b76e - unity-dev-sps-karpenter (unity|dev)
KI

In [22]:
kill_list

['i-053272eefc121963c',
 'i-0b5bad8bbb22f44b0',
 'i-0176693bd16a72214',
 'i-0d75e4a3eba3999ae',
 'i-0540726b76a61e9ef',
 'i-054791c0d8526887c']

In [23]:
save_list

['i-0bb1b86acf0dbfb26',
 'i-0fe982b45c0eff45a',
 'i-0866cb5bfdbf78adc',
 'i-0cf806754945857d6',
 'i-02be479b5204c8b4d',
 'i-0ee1f2dd75b0b15da',
 'i-07598f0de69d04498',
 'i-05dfbc59fc7ebbec5',
 'i-0db4a545e8a334023',
 'i-019104bc5ee06b76e',
 'i-015db2486b7decd67',
 'i-0b251677e5e58c030',
 'i-073a9f17094277671',
 'i-010511b9a0f5bbb01',
 'i-0fb14d92a3a730019']

In [24]:
as_client = boto3.client('autoscaling')


In [25]:
as_groups = as_client.describe_auto_scaling_instances(InstanceIds=kill_list)

In [26]:
asgs_set = set()
asgs_instance_list = []
asgs_kill_follow_up_list = []
for as_instance in as_groups['AutoScalingInstances']:
    if "eks" in as_instance['AutoScalingGroupName']:
        asgs_set.add(as_instance['AutoScalingGroupName'])
        asgs_kill_follow_up_list.append(as_instance['InstanceId'])
    asgs_instance_list.append(as_instance['InstanceId'])

In [27]:
asgs_set

set()

In [28]:
for asg_name in asgs_set:

    as_client.update_auto_scaling_group(
        AutoScalingGroupName=asg_name,
        MinSize=0,
        MaxSize=0
    )
    
    as_client.set_desired_capacity(
        AutoScalingGroupName=asg_name,
        DesiredCapacity=0,
        HonorCooldown=False
    )


In [29]:
# Get the list of instances older than 48h but _not_ in the autoscaling groups
asgs_instance_list

[]

In [30]:
kill_instance_list_not_in_auto_scaling_group = list(set(kill_list) - set(asgs_instance_list))

In [31]:
kill_instance_list_not_in_auto_scaling_group

['i-0d75e4a3eba3999ae',
 'i-053272eefc121963c',
 'i-0176693bd16a72214',
 'i-0540726b76a61e9ef',
 'i-054791c0d8526887c',
 'i-0b5bad8bbb22f44b0']

In [32]:
# Kill the autoscaling group nodes if they aren't killed yet
# This will kill any instances older than 48 hours but not in the asg_instance_lists
client.terminate_instances(
    InstanceIds=kill_instance_list_not_in_auto_scaling_group,
    DryRun = False
)

{'TerminatingInstances': [{'InstanceId': 'i-054791c0d8526887c',
   'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'PreviousState': {'Code': 48, 'Name': 'terminated'}},
  {'InstanceId': 'i-0b5bad8bbb22f44b0',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'InstanceId': 'i-053272eefc121963c',
   'CurrentState': {'Code': 48, 'Name': 'terminated'},
   'PreviousState': {'Code': 48, 'Name': 'terminated'}},
  {'InstanceId': 'i-0540726b76a61e9ef',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'InstanceId': 'i-0176693bd16a72214',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'InstanceId': 'i-0d75e4a3eba3999ae',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '0670dd5e-6d10-43d8-83ed-c44

In [14]:
# Check for the existence of 'aws:eks:cluster-name' tag in the instance to see
# if this is a karpenter node managed through EKS. If so, what do we do there? 
# Could sleep a while for the karpenter nodes to die and then shut these down.
# Better steps would be to remove the airflow deployment from 

import time
time.sleep(180) # 3 minutes

In [16]:
asgs_kill_follow_up_list

['i-053272eefc121963c', 'i-054791c0d8526887c']

In [17]:

client.terminate_instances(
    InstanceIds=asgs_kill_follow_up_list,
    DryRun = False
)

{'TerminatingInstances': [{'InstanceId': 'i-054791c0d8526887c',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}},
  {'InstanceId': 'i-053272eefc121963c',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '4ead7f8b-38ee-41b4-b756-ae0b929a6108',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4ead7f8b-38ee-41b4-b756-ae0b929a6108',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '619',
   'date': 'Mon, 12 May 2025 21:53:37 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}